In [1]:
import numpy as np
DTYPE = np.float64
# import pandas as pd
import matplotlib.pyplot as plt
from scipy.interpolate import griddata
import os
import pickle
import gzip
from tqdm import tqdm
# try:
#     import flory
    
# except ImportError:
#     print("Installing 'flory' temporarily...")
#     !pip install flory --quiet
#     import flory

# from numba import jit

In [2]:
# General Function for computing mergers of compartments
# Returns the concentrations of components in the merged and the unmerged compartments
# Stores them in phi_in_kmerged and phi_in_kunmerged respectively.


def mergers(concs:np.array, vols:np.array, chis, merged_compartments:list):
    # Find the unmerged compartment(s)
    expected = len(vols)*(len(vols)+1)//2
    actual = np.sum(merged_compartments)
    unmerged_compartment = expected - actual

    # print(concs)
    # print(vols)
    # print(unmerged)

    # Compute the merged volumes, stored in variable eta_merged
    # subtract -1 from the merged_compartments idxs to maintain python idxing
    eta_merged = 0
    for compartment in merged_compartments:
        eta_merged += vols[compartment-1]
    # print(eta_merged)

    # Calculating the compositions of components in the merged compartment
    # Taking a simple weighted average
    phi_1merged = (vols[merged_compartments[0]-1]*concs[0, merged_compartments[0]-1] + vols[merged_compartments[1]-1]*concs[0, merged_compartments[1]-1])/eta_merged
    phi_2merged = (vols[merged_compartments[0]-1]*concs[1, merged_compartments[0]-1] + vols[merged_compartments[1]-1]*concs[1, merged_compartments[1]-1])/eta_merged
    phi_3merged = 1 - phi_1merged - phi_2merged
    # print(phi_1merged, phi_2merged, phi_3merged)
    # print(phi_1merged + phi_2merged + phi_3merged)

    phi_in_kmerged = [phi_1merged, phi_2merged, phi_3merged]
    phi_in_kunmerged = [concs[0, unmerged_compartment-1], concs[1, unmerged_compartment-1], concs[2, unmerged_compartment-1]]
    # print(phi_in_kmerged)
    # print(phi_in_kunmerged)

    F_merged = eta_merged*floryHuggins(phi_in_kmerged, chis) + vols[unmerged_compartment-1]*floryHuggins(phi_in_kunmerged, chis)
    # print(F_merged)

    return phi_in_kunmerged, phi_in_kmerged, eta_merged, unmerged_compartment

    

In [3]:
# Flory Huggins Free Energy function
def floryHuggins(phi:DTYPE, chi:np.array):
    part_1 = np.sum(phi*np.log(phi))
    part_2 = 0

    for i in range(len(phi)):
        for j in range(i+1, len(phi)):
            part_2 += chi[i][j]*phi[i]*phi[j]

    return part_1 + part_2

In [4]:
phi_global = np.array([0.2, 0.2, 0.6], dtype = DTYPE)
Xs = np.arange(0, 10.1, 0.1)
tol = DTYPE(1e-4)
step_size = 0.001
n_points = 100

In [18]:
# Find the Best Merger in 2 steps:
# 1. Find the best volume slice for a specific merger from [12] [23] [13]
# 2. Then Find the best merger from these best volume slices

# I did them seprately for my convenience to debug

# Step 1
for X in (Xs):

    # Load the Flory's files to get the initial guesses to construct the perturbed state
    input_filepath = f"../../../../../../../../media/vchirag24/Shared Drive/Chirag/3_comp_3_phase/data/withVolFluctuations/phi_g{phi_global}/raw/X{X:.3f}/"
    input_filename = f"FLORY_3phase_solution.pkl"
    input_file = os.path.join(input_filepath, input_filename)
    
    with gzip.open(input_file, 'rb') as f:
        loaded_data = pickle.load(f)

    concs = loaded_data["phase_fractions"]
    vols = loaded_data["phase_volumes"]
    chis = loaded_data["chis"]

    merged_compartments_list = [np.array([1, 2], dtype=np.int64), np.array([2, 3], dtype=np.int64), np.array([1, 3], dtype=np.int64)]
    # merged_compartments_list = [np.array([1, 2], dtype=np.int64)]
    
    for idx, merged_compartments in enumerate(merged_compartments_list):
        phi_in_kunmerged, phi_in_kmerged, eta_merged, unmerged_compartment = mergers(concs, vols, chis, merged_compartments)
        
        eta_merged_perturbed = np.linspace(max(tol, eta_merged - n_points*step_size), min(1-tol, eta_merged + n_points*step_size), 2*n_points+1)

        # Find the best volume slice for a particular merger
        min_F_best_slice = DTYPE(np.inf)
        best_slice = None

        for eta_m in eta_merged_perturbed:
            input_filepath = f"../../../../../../../../media/vchirag24/Shared Drive/Chirag/3_comp_3_phase/data/withVolFluctuations/phi_g{phi_global}/raw/X{X:.3f}/mergers/{merged_compartments}/volume_slices/"
            input_filename = f"eta_m{eta_m:.3f}.pkl"
            input_file = os.path.join(input_filepath, input_filename)
            
            with gzip.open(input_file, "rb") as f:
                loaded_data = pickle.load(f)

            if len(loaded_data["F"]) > 0:
                min_F_slice = np.min(loaded_data["F"]) # minimum F in this slice
            else:
                min_F_slice = None # minimum F in this slice
            # min_F_slice = np.min(loaded_data["F"]) # minimum F in this slice
            # print(np.min(loaded_data["F"]))

            if min_F_slice is not None and (min_F_best_slice is None or min_F_slice < min_F_best_slice):
                min_F_best_slice = min_F_slice
                best_slice = eta_m
            # if min_F_slice < min_F_best_slice:
            #     min_F_best_slice = min_F_slice
            #     best_slice = eta_m
        
        # For every merger, write the best slice to a .pkl file
        output_filepath = f"../../../../../../../../media/vchirag24/Shared Drive/Chirag/3_comp_3_phase/data/withVolFluctuations/phi_g{phi_global}/analysis/X{X:.3f}/mergers/{merged_compartments}/best_volume_slice/"
        output_filename = f"best_slice.pkl"
        if not os.path.exists(output_filepath):
            os.makedirs(output_filepath)
        output_file = os.path.join(output_filepath, output_filename)
        
        data_to_save = {
            "best_slice": best_slice,
            "min_F": min_F_best_slice,
            "metadata": {"best_slice-> stores the slice per merger which returns locally the lowest free energy", 
                         "min_F-> stores the correesponding minimum F for this slice "}
            }
        
        with gzip.open(output_file, "wb") as f:
            pickle.dump(data_to_save, f, protocol=pickle.HIGHEST_PROTOCOL)
        # print(output_file, best_slice)
        # print()
        # print()

In [19]:
# Step 2
# Find the best merger
for X in (Xs):
    merged_compartments_list = [np.array([1, 2], dtype=np.int64), np.array([2, 3], dtype=np.int64), np.array([1, 3], dtype=np.int64)]
    # merged_compartments_list = [np.array([1, 2], dtype=np.int64)]

    min_F_best = DTYPE(np.inf)
    best_merger_slice = None
    best_merger_compartment = None
    for idx, merged_compartments in enumerate(merged_compartments_list):
        # Get the best slice for each merger
        input_filepath = f"../../../../../../../../media/vchirag24/Shared Drive/Chirag/3_comp_3_phase/data/withVolFluctuations/phi_g{phi_global}/analysis/X{X:.3f}/mergers/{merged_compartments}/best_volume_slice/"
        input_filename = f"best_slice.pkl"
        input_file = os.path.join(input_filepath, input_filename)
        with gzip.open(input_file, 'rb') as f:
            loaded_data = pickle.load(f)
        best_slice = loaded_data["best_slice"]
        min_F_slice = loaded_data["min_F"]

        if min_F_slice < min_F_best:
            min_F_best = min_F_slice
            best_merger_slice = best_slice
            best_merger_compartment = merged_compartments
            
    output_filepath = f"../../../../../../../../media/vchirag24/Shared Drive/Chirag/3_comp_3_phase/data/withVolFluctuations/phi_g{phi_global}/analysis/X{X:.3f}/mergers/"
    output_filename = f"best_merger.pkl"
    if not os.path.exists(output_filepath):
        os.makedirs(output_filepath)
    output_file = os.path.join(output_filepath, output_filename)
    
    data_to_save = {
        "best_merger_compartment": best_merger_compartment,
        "best_merger_slice": best_merger_slice,
        "metadata": "stores the slice in the best merged compartments which returns the lowest free energy possible"
        }
    
    with gzip.open(output_file, "wb") as f:
        pickle.dump(data_to_save, f, protocol=pickle.HIGHEST_PROTOCOL)

In [5]:
# Plotting contours for the best slices and the best merger
def plot_Contours(Xs):
    for X in (Xs):
        merged_compartments_list = [np.array([1, 2], dtype=np.int64), np.array([2, 3], dtype=np.int64), np.array([1, 3], dtype=np.int64)]
        # merged_compartments_list = [np.array([1, 2], dtype=np.int64)]
    
        # Load the best merger
        input_filepath = f"../../../../../../../../media/vchirag24/Shared Drive/Chirag/3_comp_3_phase/data/withVolFluctuations/phi_g{phi_global}/analysis/X{X:.3f}/mergers/"
        input_filename = f"best_merger.pkl"
        input_file = os.path.join(input_filepath, input_filename)
        with gzip.open(input_file, "rb") as f:
            loaded_data_best = pickle.load(f)
            
        # LOAD the best slice
        for idx, merged_compartments in enumerate(merged_compartments_list):
            # Get the best slice for each merger
            input_filepath = f"../../../../../../../../media/vchirag24/Shared Drive/Chirag/3_comp_3_phase/data/withVolFluctuations/phi_g{phi_global}/analysis/X{X:.3f}/mergers/{merged_compartments}/best_volume_slice/"
            input_filename = f"best_slice.pkl"
            input_file = os.path.join(input_filepath, input_filename)
            with gzip.open(input_file, 'rb') as f:
                loaded_data = pickle.load(f)
            best_slice = loaded_data["best_slice"]
    
    
            # Now, LOAD the phi data
            input_filepath = f"../../../../../../../../media/vchirag24/Shared Drive/Chirag/3_comp_3_phase/data/withVolFluctuations/phi_g{phi_global}/raw/X{X:.3f}/mergers/{merged_compartments}/volume_slices/"
            input_filename = f"eta_m{best_slice:.3f}.pkl"
            input_file = os.path.join(input_filepath, input_filename)
            
            with gzip.open(input_file, "rb") as f:
                loaded_data = pickle.load(f)
            # print(loaded_data["metadata"])
    
            # PLOT a contour for this loaded data
            phi1_grid = np.linspace(min(loaded_data["phi_1merged"]), max(loaded_data["phi_1merged"]), 100)
            phi2_grid = np.linspace(min(loaded_data["phi_2merged"]), max(loaded_data["phi_2merged"]), 100)
            phi1_mesh, phi2_mesh = np.meshgrid(phi1_grid, phi2_grid)
    
            # Interpolate F values onto the grid
            F_grid = griddata(
                (loaded_data["phi_1merged"], loaded_data["phi_2merged"]), 
                loaded_data["F"], 
                (phi1_mesh, phi2_mesh), 
                method='cubic'
            )
    
            # Create the contour plot
            fig, ax = plt.subplots(figsize=(4.25, 3))
            contour = plt.contourf(phi1_mesh, phi2_mesh, F_grid, levels=1000, cmap = "RdYlBu_r")

            cbar = plt.colorbar(contour, label=r'$\mathbf{F\, (k_BT)}$')
    
            # Bolden the label
            cbar.set_label(r'$\mathbf{F\, (k_BT)}$', weight='bold', fontsize=10)
            
            # Bolden the tick numbers
            cbar.ax.tick_params(
                axis='y',          # Apply to y-axis (colorbar's axis)
                direction='in',    # Ticks point inward
                length=2,          # Tick length (adjust as needed)
                width=0.5,         # Tick width (optional)
                labelsize=8,      # Tick label size
            )
            for label in cbar.ax.get_yticklabels():
                label.set_weight('bold')
                    
            # Mark the minimum value of F
            min_F_index = np.argmin(loaded_data["F"])
            min_phi1 = loaded_data["phi_1merged"][min_F_index]
            min_phi2 = loaded_data["phi_2merged"][min_F_index]
            min_F = loaded_data["F"][min_F_index]
    
            ax.scatter(
                min_phi1, min_phi2, 
                color='lime', 
                s=10,
                marker = "*",
                linewidth=0.5,
                # label=label
            )
            
            
            # Add contour lines
            CS = plt.contour(phi1_mesh, phi2_mesh, F_grid, levels=40, colors='k', linewidths=0.25)
            
            phi1_d = phi1_grid[1]-phi1_grid[0]
            phi2_d = phi2_grid[1]-phi2_grid[0]
            ax.set_xlim([phi1_grid[0]-2*phi1_d, phi1_grid[-1]+2*phi1_d])
            ax.set_ylim([phi2_grid[0]-2*phi2_d, phi2_grid[-1]+2*phi2_d])
            
            
            ax.set_xlabel(r'$\mathbf{\phi_{1, {merged}}}$', fontsize=12)
            ax.set_ylabel(r'$\mathbf{\phi_{2, {merged}}}$', fontsize=12)
            ax.tick_params(axis='both', which='both', direction='in', width=1.5, length=3.5, labelsize=10)
            ax.tick_params(axis='both', which='minor', direction='in', width=0.5, length=2, labelsize=8)
            for label in ax.get_xticklabels() + ax.get_yticklabels():
                label.set_fontweight("bold")

            # title = f"X = {X:.3f}" + "\n" + r"$\Phi^{\text{global}}=$" + f"{phi_global}" +"\n" + f'Merged compartments: {loaded_data["metadata"]["merged_compartments"]}' + "\n" + r"$\eta^{\text{merged}}=$" + f"{best_slice:.3f}"
            # plt.title(title)
    
            
            # phi1_guess = loaded_data["metadata"]["initial_merged_concentration_guess"][0]
            # phi2_guess = loaded_data["metadata"]["initial_merged_concentration_guess"][1]
            # suptitle = r"($\phi_{1, \text{merged}}^{in}, \phi_{2, \text{merged}}^{in}) =$" + f"({phi1_guess:.3f}, {phi2_guess:.3f})"
            # fig.suptitle(suptitle)
            
            # plt.legend(loc = "upper right")
            # legend = ax.legend(loc='upper center', fontsize=6, frameon=True)
            # for text in legend.get_texts():
            #     text.set_fontweight("bold") 

            # title = r"($\mathbf{\phi_{1, {merged}}^{{best}}, \phi_{2, {merged}}^{{best}}}) \mathbf{=}$" + f"({min_phi1:.3f}, {min_phi2:.3f}, {best_slice:.3f})" + "\n" + r"$\mathbf{\eta_{merged}^{{best}}}$"
            # fig.suptitle(title, fontweight="bold")
            
            fig.tight_layout()            
            input_filepath = f"../../../../../../../../media/vchirag24/Shared Drive/Chirag/3_comp_3_phase/data/withVolFluctuations/phi_g{phi_global}/raw/X{X:.3f}/mergers/{merged_compartments}/volume_slices/"
            input_filename = f"eta_m{best_slice:.3f}.pkl"
            input_file = os.path.join(input_filepath, input_filename)
            
            with gzip.open(input_file, "rb") as f:
                loaded_data = pickle.load(f)
            # print(best_slice)
            output_filepath = f"../../../../../../../../media/vchirag24/Shared Drive/Chirag/3_comp_3_phase/data/withVolFluctuations/phi_g{phi_global}/analysis/X{X:.3f}/mergers/{merged_compartments}/best_volume_slice/"
            output_filename = f"({min_phi1:.3f}, {min_phi2:.3f}, {best_slice:.3f}).png"
            if not os.path.exists(output_filepath):
                os.makedirs(output_filepath)
            output_file = os.path.join(output_filepath, output_filename)
            
            plt.savefig(output_file, dpi=400)
            # plt.close()
    
            if np.array_equal(merged_compartments, loaded_data_best["best_merger_compartment"]):
                output_filepath = f"../../../../../../../../media/vchirag24/Shared Drive/Chirag/3_comp_3_phase/data/withVolFluctuations/phi_g{phi_global}/analysis/X{X:.3f}/mergers/"
                output_filename = f"({min_phi1:.3f}, {min_phi2:.3f}, {best_slice:.3f})_best.png"
                if not os.path.exists(output_filepath):
                    os.makedirs(output_filepath)
                output_file = os.path.join(output_filepath, output_filename)
                
                plt.savefig(output_file, dpi=400)
            plt.close()

In [6]:
plot_Contours(Xs)

Locator attempting to generate 1000 ticks ([-0.050850000000000006, ..., 0.099]), which exceeds Locator.MAXTICKS (1000).
Locator attempting to generate 1003 ticks ([-0.023100000000000002, ..., 0.2775]), which exceeds Locator.MAXTICKS (1000).
Locator attempting to generate 1001 ticks ([-0.022000000000000002, ..., 0.378]), which exceeds Locator.MAXTICKS (1000).
Locator attempting to generate 1002 ticks ([0.4905, ..., 0.9910000000000001]), which exceeds Locator.MAXTICKS (1000).
Locator attempting to generate 1000 ticks ([-0.021, ..., 0.978]), which exceeds Locator.MAXTICKS (1000).
Locator attempting to generate 1000 ticks ([-0.0216, ..., 0.5778000000000001]), which exceeds Locator.MAXTICKS (1000).
Locator attempting to generate 1000 ticks ([0.6162000000000001, ..., 1.2156000000000002]), which exceeds Locator.MAXTICKS (1000).
Locator attempting to generate 1000 ticks ([0.6240000000000001, ..., 1.2234000000000003]), which exceeds Locator.MAXTICKS (1000).
